## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-28-19-52-02 +0000,bbc,Moldovans await pivotal election result as lea...,https://www.bbc.com/news/articles/cx2rdlj8ejgo...
1,2025-09-28-19-49-12 +0000,nyt,"Shooting at Mormon Church in Grand Blanc, Mich...",https://www.nytimes.com/live/2025/09/28/us/gra...
2,2025-09-28-19-48-52 +0000,nypost,Iraq War veteran Thomas Sanford ID’d as gunman...,https://nypost.com/2025/09/28/us-news/iraq-war...
3,2025-09-28-19-48-51 +0000,nyt,Afghanistan Frees a U.S. Citizen Held in Prison,https://www.nytimes.com/2025/09/28/world/asia/...
4,2025-09-28-19-47-00 +0000,wsj,Opinion | Green Energy Isn’t as Affordable as ...,https://www.wsj.com/opinion/green-energy-isnt-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2454/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
73,trump,29
9,shooting,12
11,church,11
79,new,9
44,dead,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
57,2025-09-28-15-55-00 +0000,wsj,President Trump has agreed to meet with the fo...,https://www.wsj.com/politics/policy/trump-to-h...,76
59,2025-09-28-15-50-50 +0000,nypost,Grand Blanc Township Mormon church shooting le...,https://nypost.com/2025/09/28/us-news/michigan...,70
13,2025-09-28-19-23-11 +0000,bbc,At least two dead and several injured in shoot...,https://www.bbc.com/news/articles/ceq2vd15glwo...,63
35,2025-09-28-17-42-29 +0000,wapo,"At least one dead, nine injured in Michigan ch...",https://www.washingtonpost.com/nation/2025/09/...,62
54,2025-09-28-16-05-59 +0000,nypost,Protesters clash with ICE agents as Trump depl...,https://nypost.com/2025/09/28/us-news/proteste...,61


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
57,76,2025-09-28-15-55-00 +0000,wsj,President Trump has agreed to meet with the fo...,https://www.wsj.com/politics/policy/trump-to-h...
59,70,2025-09-28-15-50-50 +0000,nypost,Grand Blanc Township Mormon church shooting le...,https://nypost.com/2025/09/28/us-news/michigan...
34,57,2025-09-28-17-44-38 +0000,nypost,Eric Adams rips political ‘extremism’ as he dr...,https://nypost.com/2025/09/28/us-news/eric-ada...
54,32,2025-09-28-16-05-59 +0000,nypost,Protesters clash with ICE agents as Trump depl...,https://nypost.com/2025/09/28/us-news/proteste...
35,30,2025-09-28-17-42-29 +0000,wapo,"At least one dead, nine injured in Michigan ch...",https://www.washingtonpost.com/nation/2025/09/...
93,30,2025-09-28-13-13-51 +0000,bbc,Building of three new towns will start before ...,https://www.bbc.com/news/articles/cly1geen679o...
89,28,2025-09-28-13-21-43 +0000,nypost,Southport mass shooting suspect identified as ...,https://nypost.com/2025/09/28/us-news/southpor...
145,25,2025-09-28-03-29-21 +0000,nypost,Rep. Marjorie Taylor Greene warns in X rant t...,https://nypost.com/2025/09/27/us-news/rep-marj...
123,24,2025-09-28-09-08-40 +0000,nypost,"Russia fires over 600 drones, missiles in dead...",https://nypost.com/2025/09/28/world-news/large...
60,21,2025-09-28-15-48-56 +0000,nypost,James Comey’s ex-No. 2 gives mealy-mouthed ans...,https://nypost.com/2025/09/28/us-news/comeys-e...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
